In [3]:
from keras import Sequential
from keras.layers import Dense, Dropout, LSTM

Using TensorFlow backend.


In [4]:
import pandas as pd
import numpy as np
import ast

In [20]:
nepse_data = pd.read_csv("data_set_ready_for_training.csv")
nepse_data["news"] = nepse_data["news"].apply(ast.literal_eval)
nepse_data = nepse_data.replace([np.inf, -np.inf], np.nan).dropna()

In [22]:
nepse_data["Label"] = nepse_data["closing_price"].rolling(window=2).apply(lambda x:  1 if x[1]>x[0] else 0 )
nepse_data.Label = nepse_data.Label.shift(-1)
nepse_data = nepse_data[:-1]
nepse_data.tail()

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


,closing_price,news,momentum,EMA,MACD,RSI,ROI,ATR,williams,CCI,UO,Label
1372,1273.0,"[[0.36220553517341614, 0.9683135151863098, 0.5...",73.0,1257.027190,-82.501011,5.000000,0.967400,85.0,0.141176,1252.666667,0.517338,1.0
1373,1277.0,"[[0.38373106718063354, 0.8309400081634521, 0.5...",77.0,1260.099930,-74.796552,7.416667,0.965309,85.0,0.094118,1254.000000,0.495022,0.0
1374,1272.0,"[[0.3729100525379181, 0.8552981615066528, 0.60...",53.0,1261.930710,-68.306774,5.235294,0.971226,66.0,0.196970,1258.666667,0.489393,0.0
1375,1265.0,"[[0.15445244312286377, 0.6669142842292786, 0.7...",-20.0,1262.402908,-63.002169,2.916667,0.980316,20.0,1.000000,1271.666667,0.453701,0.0
1376,1257.0,"[[0.3814601004123688, 0.7400802969932556, 0.64...",-16.0,1261.571692,-58.766343,0.125000,0.990692,20.0,1.000000,1263.666667,0.465451,0.0


In [23]:
data = nepse_data.loc[:, ["closing_price", "momentum","EMA","MACD","RSI","ROI","ATR","williams",
                                "CCI","UO","Label"]].values

In [24]:
look_back = 40
forward_days = 1
num_periods = 20
batch_size = 1

In [28]:
import numpy as np
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(look_back,len(dataset)-1):
        a = dataset[(i) : i - look_back : -1,:10]
        dataX.append(list(reversed(a)))
        dataY.append(dataset[i+1,10])
    return np.array(dataX), np.array(dataY)

In [29]:
X, y = create_dataset(data, look_back=look_back )

In [30]:
X[-1],y[-1]

(array([[ 1.75000000e+03,  2.20000000e+01,  1.69889004e+03,
          5.77275023e+01,  3.76666667e+00,  9.69485714e-01,
          5.20000000e+01,  5.76923077e-01,  1.75266667e+03,
          8.24229401e-01],
        [ 1.72000000e+03, -4.80000000e+01,  1.70213773e+03,
          5.51353761e+01,  8.66666667e-01,  9.92209302e-01,
          6.00000000e+01,  1.00000000e+00,  1.74000000e+03,
          7.18564310e-01],
        [ 1.71800000e+03, -6.20000000e+01,  1.70457808e+03,
          5.23166405e+01,  1.93548387e-01,  9.98952270e-01,
          6.20000000e+01,  1.00000000e+00,  1.73866667e+03,
          6.24764099e-01],
        [ 1.73700000e+03, -2.10000000e+01,  1.70956607e+03,
          5.10276979e+01,  3.06451613e-01,  9.93897524e-01,
          4.00000000e+01,  5.25000000e-01,  1.73766667e+03,
          6.68803539e-01],
        [ 1.73500000e+03, -1.50000000e+01,  1.71347898e+03,
          4.92767862e+01,  4.52380952e-01,  1.00063401e+00,
          3.20000000e+01,  4.68750000e-01,  1.734333

In [31]:

fact = 0.1
total = len(X)
factor = int((total * (1-fact))/batch_size)*batch_size
X_train, X_validate = X[:factor] , X[factor:int(total/batch_size)*batch_size] 
y_train, y_validate = y[:factor] , y[factor:int(total/batch_size)*batch_size] 

In [32]:
y_validate.shape

(128,)

In [36]:
NUM_NEURONS_FirstLayer = 128
NUM_NEURONS_SecondLayer = 64
EPOCHS = 220

data_dim = 10
#Build the model
model = Sequential()
model.add(LSTM(NUM_NEURONS_FirstLayer,batch_input_shape=(batch_size,look_back,data_dim), return_sequences=True,stateful=True))
model.add(LSTM(NUM_NEURONS_SecondLayer,stateful=True))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,y_train,epochs=EPOCHS,validation_data=(X_validate,y_validate),shuffle=False,batch_size=batch_size, verbose=2)

Train on 1148 samples, validate on 128 samples
Epoch 1/220
 - 42s - loss: 0.7035 - acc: 0.5235 - val_loss: 0.6925 - val_acc: 0.5625
Epoch 2/220
 - 56s - loss: 0.6955 - acc: 0.5305 - val_loss: 0.6910 - val_acc: 0.5625
Epoch 3/220
 - 41s - loss: 0.6957 - acc: 0.5157 - val_loss: 0.6886 - val_acc: 0.5625
Epoch 4/220
 - 41s - loss: 0.6895 - acc: 0.5505 - val_loss: 0.6885 - val_acc: 0.5625
Epoch 5/220
 - 41s - loss: 0.6904 - acc: 0.5279 - val_loss: 0.6866 - val_acc: 0.5625
Epoch 6/220
 - 42s - loss: 0.6930 - acc: 0.5235 - val_loss: 0.6884 - val_acc: 0.5625
Epoch 7/220
 - 42s - loss: 0.6898 - acc: 0.5444 - val_loss: 0.6872 - val_acc: 0.5625
Epoch 8/220
 - 42s - loss: 0.6899 - acc: 0.5314 - val_loss: 0.6858 - val_acc: 0.5625
Epoch 9/220
 - 41s - loss: 0.6887 - acc: 0.5488 - val_loss: 0.6858 - val_acc: 0.5625
Epoch 10/220
 - 43s - loss: 0.6882 - acc: 0.5497 - val_loss: 0.6855 - val_acc: 0.5625
Epoch 11/220
 - 43s - loss: 0.6872 - acc: 0.5523 - val_loss: 0.6856 - val_acc: 0.5625
Epoch 12/220
 - 

In [214]:
model.predict(X_validate) 

InvalidArgumentError: PartialTensorShape: Incompatible shapes during merge: [32,128] vs. [1,128]
	 [[{{node lstm_45/TensorArrayStack/TensorArrayGatherV3}}]]